In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from src.io.datasets import CSVReader
from src.config.settings import get_settings

%matplotlib widget

In [2]:
settings = get_settings()

reader = CSVReader()

In [3]:
disguise_results = reader.read(settings.WEIGHTED_PLAY_DISGUISE_RESULTS_FILE)

In [4]:
blitz_results = disguise_results[disguise_results["wasInitialPassRusher"] == 1]

In [5]:
# Only players with more than X blitzes
play_counts = blitz_results.groupby('nflId').size()
players_with_enough_plays = play_counts[play_counts > 5].index
filtered_data = blitz_results[blitz_results['nflId'].isin(players_with_enough_plays)]

In [6]:
filtered_data = filtered_data.copy()
filtered_data['mean_disguise_score'] = filtered_data.groupby('nflId')['disguise_score'].transform('mean')

df_sum = filtered_data.groupby('nflId')[['wasInitialPassRusher', 'quarterbackHit', 'causedPressure', 'tackleForALoss']].sum().reset_index()
filtered_data_unique = filtered_data.drop_duplicates(subset=['nflId'])

df_combined = pd.merge(filtered_data_unique, df_sum, on='nflId', how='left', suffixes=('', '_sum'))

df_sorted = df_combined.sort_values(by='mean_disguise_score', ascending=False)

In [7]:
df_final = df_sorted[['jerseyNumber', 'club', 'position', 'displayName', 'mean_disguise_score', 'wasInitialPassRusher_sum', 'quarterbackHit_sum', 'causedPressure_sum', 'tackleForALoss_sum']]
df_final = df_final.rename(columns={
    'position': 'Position',
    'jerseyNumber': 'Number',
    'displayName': 'Name',
    'club': 'Team',
    'mean_disguise_score': 'Disguise Score',
    'wasInitialPassRusher_sum': 'Number of Blitzes',
    'quarterbackHit_sum': 'QB Hits',
    'causedPressure_sum': 'Pressures',
    'tackleForALoss_sum': 'Tackles for Loss'
})
df_final = df_final.reset_index(drop=True)

In [8]:
df_final

,Number,Team,Position,Name,Disguise Score,Number of Blitzes,QB Hits,Pressures,Tackles for Loss
0,19,NO,CB,Chris Harris,0.925450,11,0,3,0
1,21,DEN,CB,K'Waun Williams,0.917675,6,0,2,0
2,38,KC,CB,L'Jarius Sneed,0.914792,10,1,2,0
3,30,TB,CB,Dee Delaney,0.905634,9,0,1,0
4,30,NYJ,CB,Michael Carter,0.856780,7,0,2,0
5,3,NE,SS,Jabrill Peppers,0.851572,7,0,1,0
6,26,SEA,SS,Ryan Neal,0.836409,10,1,2,1
7,2,JAX,SS,Rayshawn Jenkins,0.833038,11,0,3,0
8,27,DAL,FS,Jayron Kearse,0.795592,11,1,2,1
9,3,ARI,SS,Budda Baker,0.779986,17,1,3,0


In [9]:
from plottable import ColumnDefinition, Table
from plottable.plots import image
from plottable.cmap import normed_cmap
from matplotlib.colors import LinearSegmentedColormap

df = df_final.copy()

df = df[['Name', 'Position', 'Team', 'Disguise Score', 'Number of Blitzes', 'QB Hits', 'Pressures', 'Tackles for Loss']]

team_abbreviations = df['Team'].unique()

team_logos = {team: f"../assets/logos/{team}.png" for team in team_abbreviations}

df['Team'] = df['Team'].map(team_logos)
df = df.set_index("Name")
df['Disguise Score'] = df['Disguise Score'].round(4)

cmap = LinearSegmentedColormap.from_list(
    name="bugw", colors=["#ffffff", "#f2fbd2", "#c9ecb4", "#93d3ab", "#35b0ab"], N=256
)
stats_cmap = LinearSegmentedColormap.from_list(
    name="stats_cmap", colors=["#cfe2f3", "#f9f2a4", "#7dbf7d"], N=256
)

name_column = ColumnDefinition("Name", textprops={"ha": "left"}, group="Player")
position_column = ColumnDefinition("Position", textprops={"ha": "center"}, group="Player")
team_column = ColumnDefinition("Team", plot_fn=image, textprops={"ha": "center"}, group="Player")

number_of_blitzes_column = ColumnDefinition("Number of Blitzes", border='l', group="Statistics")
qb_hits_column = ColumnDefinition("QB Hits", cmap=normed_cmap(df["QB Hits"], cmap=stats_cmap), group="Statistics")
pressures_column = ColumnDefinition("Pressures", cmap=normed_cmap(df["Pressures"], cmap=stats_cmap),group="Statistics")
tackles_for_loss_column = ColumnDefinition("Tackles for Loss", cmap=normed_cmap(df["Tackles for Loss"], cmap=stats_cmap),group="Statistics")

disguise_column = ColumnDefinition("Disguise Score",
            cmap=normed_cmap(df["Disguise Score"], cmap=cmap), border='l')

fig, ax = plt.subplots(figsize=(12, 9))
plt.tight_layout(pad=0)
table = Table(df, column_definitions=[name_column, position_column, team_column, disguise_column, number_of_blitzes_column, qb_hits_column, pressures_column, tackles_for_loss_column])
fig.savefig("../reports/figures/table_2.png", facecolor=ax.get_facecolor(), dpi=200, bbox_inches='tight')
plt.close(fig)